In [4]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import os

# Cargar archivo
file_path = "Actualización Censo Resguardo Indigena de Guachucal - 2025 (1).xlsx"
df = pd.read_excel(file_path, sheet_name=0, skiprows=6)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# Eliminar filas vacías
df = df.dropna(subset=['FAMILIA', 'INTEGRANTES'])

# Convertir a enteros
df['FAMILIA'] = df['FAMILIA'].astype(int)   
df['INTEGRANTES'] = df['INTEGRANTES'].astype(int)

# Reasignar códigos de familia
nuevo_codigo = []
codigo_actual = 0  # Ahora empieza en 0 para que el primer grupo sea 1
ultimo_codigo = None

for _, row in df.iterrows():
    if row['FAMILIA'] != ultimo_codigo:
        codigo_actual += 1  # Aquí se incrementa solo cuando cambia la familia
        ultimo_codigo = row['FAMILIA']

    nuevo_codigo.append(codigo_actual)

df['Nuevo_Codigo_Familia'] = nuevo_codigo

# Verificar que el número de integrantes sea correcto
df['Validacion'] = df.groupby('Nuevo_Codigo_Familia')['Nuevo_Codigo_Familia'].transform('count')
df['Error'] = df['Validacion'] != df['INTEGRANTES']
df['Error'] = df['Error'].replace({True: 'ERROR - Revisar', False: ''})

# Definir nombre del archivo de salida
nombre_archivo = os.path.splitext(file_path)[0]
output_file = f"VALIDADO_{nombre_archivo}.xlsx"

# Guardar archivo con formato en Excel
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Censo Ordenado", index=False)
    workbook = writer.book
    worksheet = writer.sheets["Censo Ordenado"]

# Aplicar color a toda la fila donde haya error
workbook = load_workbook(output_file)
worksheet = workbook["Censo Ordenado"]

fill = PatternFill(start_color="FF9999", end_color="FF9999", fill_type="solid")

for row_idx, error_value in enumerate(df['Error'], start=2):  # Saltamos el encabezado
    if error_value == "ERROR - Revisar":
        for col_idx in range(1, len(df.columns) + 1):  # Recorrer todas las columnas
            worksheet.cell(row=row_idx, column=col_idx).fill = fill

# Guardar cambios en el archivo Excel
workbook.save(output_file)
print(f"Archivo guardado en {output_file} con formato de color aplicado en filas con error.")


Archivo guardado en VALIDADO_Actualización Censo Resguardo Indigena de Guachucal - 2025 (1).xlsx con formato de color aplicado en filas con error.
